# Required External Libraries

In [0]:
 # Auto Machine Learning Library
!pip install tpot
!pip install mlbox
!pip install h2o

# Python package for stacking (machine learning technique)
!pip install vecstack 
!pip install mlxtend

# Model Interpretation Libraries
!pip -q install shap
!pip -q install lime
!pip -q install eli5
!pip -q install yellowbrick

In [0]:
# Basic Library for Data Manupulation
import pandas as pd
import numpy as np
import random as rnd

# Reading Diffrent types of data format
import pyreadstat 
import sqlite3
from pandas.io import sql
import pyreadr


#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline

# Importing Models
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# Importing other tools
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.metrics import accuracy_score, recall_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.externals import joblib


# Ignoring Warnings
import warnings
warnings.filterwarnings('ignore')

# External imports
from vecstack import stacking
from tpot import TPOTClassifier

from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

import h20
from h2o.automl import H2OAutoML

import eli5
from eli5.sklearn import PermutationImportance

import shap

import lime

from yellowbrick.features import Rank2D
from yellowbrick.classifier import ClassificationReport

# Loading Data

- Data can be in any of the popular formats - CSV, TXT, XLS/XLSX (Excel), sas7bdat (SAS), Stata, Rdata (R) etc.

In [0]:
# Import CSV files
Data= pd.read_csv("PATH_TO_FILE/example.csv")
# Import File from URL
Data = pd.read_csv("URL_FOR_THE_FILE")
# Read Text File
Data = pd.read_table("PATH_TO_FILE/example.txt")
Data = pd.read_csv("PATH_TO_FILE/example.txt", sep ="\t")
# Read Excel File
Data = pd.read_excel("PATH_TO_FILE(or)URL_FOR_THE_FILE/example.xls",sheetname="Name")
#  Read SAS File
Data = pd.read_sas('example.sas7bdat')
# Read Stata File
Data = pd.read_stata('example.dta')
# Read R Data File
result = pyreadr.read_r('C:/Users/sampledata.RData')
print(result.keys()) # let's check what objects we got
Data = result["df1"] # extract the pandas data frame for object df1
# Read SQL Table from Sqlite3 with .db exension
conn = sqlite3.connect('C:/Users/Deepanshu/Downloads/flight.db')
query = "SELECT * FROM flight"
Data = pd.read_sql(query, con=conn)
# Read Data from SPSS File
Data, meta = pyreadstat.read_sav("file.sav", apply_value_formats=True)

Modules to be used based on size of the data

- Pandas - small DataSets upto 1 GB
- Dask - Medium DataSets upto XX GB
- Vaex - Large DataSets upto TB's

In [0]:
# Example

data = pd.read_csv("https://raw.githubusercontent.com/Mineria/Titanic/master/csv/train.csv")

# Preprocessing



Basic preprocessing Steps:

- Classifying the dependent and Independent Variables
- Dealing with Missing Data
- Dealing with Categorical Data
- Splitting the Dataset into Training and Testing sets
- Scaling the features

In [0]:
# Example Preprocessing

# Drop Ticket & Cabin 
data = data.drop(['Ticket', 'Cabin'], axis=1)

# Get the title from name
data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
data['Title'] = data['Title'].map(title_mapping)
data = data.drop(['Name', 'PassengerId'], axis=1)

#Change Sex to Numeric
data['Sex'] = data['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

#Add Family Size 
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1
data = data.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)

#Imputing Missing Value 
data['Age'].fillna(data['Age'].dropna().median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].dropna().mode()[0], inplace=True)

# Categorizing Numerical Value 
data['FareBand'] = pd.qcut(data['Fare'], 4).astype(str)
data['AgeBand'] = pd.qcut(data['Age'], 4).astype(str)
data = data.drop(['Fare', 'Age'], axis=1)

# Converting Embark to Number
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# 
data['FareBand'] = data['FareBand'].map( {'(-0.001, 7.91]': 0, '(31.0, 512.329]': 3, '(7.91, 14.454]': 1, '(14.454, 31.0]':2 } ).astype(int)
data['AgeBand'] = data['AgeBand'].map( {'(0.419, 22.0]': 0, '(35.0, 80.0]': 3, '(22.0, 28.0]': 1, '(28.0, 35.0]':2 } ).astype(int)

# Change in the cateogry Type & Data Columns to make it dummy
df = data.drop(['Survived'], axis=1)
df2 = df
for col in df.columns:
    df2[col] = df[col].astype('category')
  

# Model Selection

## Simple Block to experiment all available Algorithm without setting any parameter

#### Baseline Model Selection

In [0]:
Target = ['Survived']
data1_x_bin = pd.get_dummies(df2)

# X = data1_x_bin
# y = Target
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]

#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = data[Target]  # Y 

#index through MLA and save performance to table
row_index = 0
Feature_Importance = {}

for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, data1_x_bin, data[Target], cv  = cv_split,return_train_score=True,scoring='f1')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(data1_x_bin, data[Target])

    try:
      Feature_Importance[MLA_name] = alg.feature_importances_
    except AttributeError:
      pass
      
    MLA_predict[MLA_name] = alg.predict(data1_x_bin)
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
MLA_compare['Difference'] = (MLA_compare['MLA Test Accuracy Mean']-MLA_compare['MLA Train Accuracy Mean'])*100
MLA_compare

#MLA_predict

In [0]:
# sns.lineplot(x="MLA Train Accuracy Mean", y="MLA Test Accuracy Mean", hue="MLA Name", data=MLA_compare)
from matplotlib import pyplot as plt
plt.figure(figsize=(18,8))
cmap = sns.cubehelix_palette(dark=.1, light=0.8, as_cmap=True)
# cmap = sns.palplot(sns.hls_palette(8, l=.3, s=.8))
sns.scatterplot(x="MLA Train Accuracy Mean", y="MLA Name", data=MLA_compare,hue="MLA Time",palette=cmap ,sizes=(20, 200))

In [0]:
# sns.lineplot(x="MLA Train Accuracy Mean", y="MLA Test Accuracy Mean", hue="MLA Name", data=MLA_compare)
from matplotlib import pyplot as plt
plt.figure(figsize=(18,8))
cmap = sns.cubehelix_palette(dark=.1, light=0.8, as_cmap=True)
# cmap = sns.palplot(sns.hls_palette(8, l=.3, s=.8))
sns.scatterplot(x="MLA Test Accuracy Mean", y="MLA Name", data=MLA_compare,hue="MLA Time",palette=cmap ,sizes=(20, 200))

In [0]:
# sns.lineplot(x="MLA Train Accuracy Mean", y="MLA Test Accuracy Mean", hue="MLA Name", data=MLA_compare)
from matplotlib import pyplot as plt
plt.figure(figsize=(18,8))
cmap = sns.cubehelix_palette(dark=.1, light=0.8, as_cmap=True)
# cmap = sns.palplot(sns.hls_palette(8, l=.3, s=.8))
sns.scatterplot(x="Difference", y="MLA Name", data=MLA_compare,hue="MLA Time",palette=cmap ,sizes=(20, 200))

From the above results Decide the base models to tune it furthur

#### Feature Selection

In [0]:
# Get Importance Feature 

feature_names = data1_x_bin.columns
feat_imp_df = pd.DataFrame.from_dict(Feature_Importance)
feat_imp_df.index = feature_names
feat_imp_df

In [0]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

scaled_fi = pd.DataFrame(data=mms.fit_transform(feat_imp_df),
                         columns=feat_imp_df.columns,
                         index=feat_imp_df.index)

scaled_fi['Overall'] = scaled_fi.sum(axis=1)
print(scaled_fi.head())
ordered_ranking = scaled_fi.sort_values('Overall', ascending=False)
fig, ax = plt.subplots(figsize=(10,7), dpi=80)
sns.barplot(data=ordered_ranking, y=ordered_ranking.index, x='Overall', palette='magma')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.xaxis.set_visible(False)
ax.grid(False)
ax.set_title('Feature Importances for all Models');

Decide the number of fetures you want to take in your model

In [0]:
# Re-Building the model with only top 10 Features 

scaled_sorted = scaled_fi.sort_values(by = ['Overall'], ascending = False)
scaled_sorted[:5].index
important_col = list(scaled_sorted[:10].index)
Target = ['Survived']
data1_x_bin = pd.get_dummies(df2)
data1_x_bin = data1_x_bin[important_col]

# X = data1_x_bin
# y = Target
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]

#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = data[Target]

#index through MLA and save performance to table
row_index = 0
Feature_Importance = {}

for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, data1_x_bin, data[Target], cv  = cv_split,return_train_score=True,scoring='f1')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(data1_x_bin, data[Target])

    try:
      Feature_Importance[MLA_name] = alg.feature_importances_
    except AttributeError:
      pass
      
    MLA_predict[MLA_name] = alg.predict(data1_x_bin)
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
MLA_compare['Difference'] = (MLA_compare['MLA Test Accuracy Mean']-MLA_compare['MLA Train Accuracy Mean'])*100
MLA_compare

From the results again decide the models for furthur tuning

## Grid Search

In [0]:
# Helper Class for Initilizing GridSearch

import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        self.best_params = {}
        self.feature_importance = {}
        self.FeatureImportanceAlgo = ['DecisionTreeClassifier','RandomForestClassifier','ExtraTreesClassifier','GradientBoostingClassifier']

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=True):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs  
            self.best_params[key]  = str(gs.best_params_)
            if key in self.FeatureImportanceAlgo:
              self.feature_importance[key]= gs.best_estimator_ .feature_importances_

            # print (gs.best_params_.feature_importances_ )
            # try:
            #   print(gs.best_params_.feature_importances_ )
            #   self.feature_importance[key]= gs.best_params_.feature_importances_ 
            # except AttributeError:
            #   pass

    def returnBestParamDF(self):
      d = self.best_params
      BestParamDF = pd.DataFrame.from_dict([d.keys(), d.values()]).T
      return BestParamDF

    # def Feature_Importance(self):
    #   for each

    # def returnFeatureImportance(self):


    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

models1 = {
    
    'LogisticRegression':LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNNClassifier': KNeighborsClassifier(),
    'SVC': SVC(),
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
    
}

params1 = {
    'LogisticRegression': { "C":np.logspace(-3,3,7),
                           "penalty":["l1","l2"]
                           },
    'DecisionTreeClassifier': {'criterion' : ['gini', 'entropy'],
                               'splitter' : ['random', 'best'],
                               'max_depth':[2,5,10], 
                               'min_samples_leaf':[2,5,10]
                               },
    'RandomForestClassifier': { 'n_estimators': [16, 32] 
                               },
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] 
                             },
    'KNNClassifier':{ 'n_neighbors': [5,10,15,20],
                     'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
                     },
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] 
                            },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32],
                                   'learning_rate': [0.8, 1.0]
                                   },
    'SVC': [
            {'kernel': ['linear'],
             'C': [1, 10]
             },
            {'kernel': ['rbf'], 
             'C': [1, 10],
             'gamma': [0.001, 0.0001]
             },
            ]
}

- Add as many parameters required for tuning thats upto You

#### Model Selection

In [0]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(data1_x_bin, data[Target], scoring='f1', n_jobs=-1)

# To run with important column
# ImpCol = ['','']
# helper1.fit(data1_x_bin[ImpCol], data[Target], scoring='f1', n_jobs=-1)

#### Feature Selection

In [0]:
feature_names = data1_x_bin.columns
feat_imp_df = pd.DataFrame.from_dict(helper1.feature_importance)
feat_imp_df.index = feature_names
feat_imp_df

In [0]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

scaled_fi = pd.DataFrame(data=mms.fit_transform(feat_imp_df),
                         columns=feat_imp_df.columns,
                         index=feat_imp_df.index)

scaled_fi['Overall'] = scaled_fi.sum(axis=1)
print(scaled_fi.head())
ordered_ranking = scaled_fi.sort_values('Overall', ascending=False)
fig, ax = plt.subplots(figsize=(10,7), dpi=80)
sns.barplot(data=ordered_ranking, y=ordered_ranking.index, x='Overall', palette='magma')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.xaxis.set_visible(False)
ax.grid(False)
ax.set_title('Feature Importances for all Models');

In [0]:
# Evalution Metrics for all Algorithms
helper1.score_summary(sort_by='max_score')

In [0]:
# Getting the Best Parameter
helper1.returnBestParamDF()

# Ensemble Learning


Ensembles combine predictions from different models to generate a final prediction, and the more models we include the better it performs.

![](https://www.dataquest.io/wp-content/uploads/2019/01/ensemble_network.png)

[Source](http://flennerhag.com/2017-04-18-introduction-to-ensembles/)

Some of the Ensemble Learning Techniques are

- Basic Ensemble Techniques
--  Max Voting
--  Averaging
--  Weighted Average
- Advanced Ensemble Techniques
--  Stacking
--  Blending
--  Bagging
--  Boosting
- Algorithms based on Bagging and Boosting
--  Bagging meta-estimator
--  Random Forest
--  AdaBoost
--  GBM
--  XGB
--  Light GBM
--  CatBoost


[Source](https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/)


## Max Voting

In [0]:
vote_est = [
    #Ensemble Methods: http://scikit-learn.org/stable/modules/ensemble.html
    
    ('rfc', ensemble.RandomForestClassifier()),
    
    #Nearest Neighbor: http://scikit-learn.org/stable/modules/neighbors.html
    ('knn', neighbors.KNeighborsClassifier()),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
   ('xgb', XGBClassifier()),
   ('lgbm',LGBMClassifier())

]

seed = 123
skf = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = seed )
#Hard Vote or majority rules
vote_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
vote_hard_cv = model_selection.cross_validate(vote_hard, data1_x_bin, data[Target], cv  = skf,scoring='f1')
vote_hard.fit(data1_x_bin, data[Target])
#print("Hard Voting Training w/bin score mean: {:.2f}". format(vote_hard_cv['train_score'].mean()*100)) 
print("Hard Voting Test w/bin score mean: {:.2f}". format(vote_hard_cv['test_score'].mean()*100))
print("Hard Voting Test w/bin score 3*std: +/- {:.2f}". format(vote_hard_cv['test_score'].std()*100*3))
print('-'*10)


#Soft Vote or weighted probabilities
vote_soft = ensemble.VotingClassifier(estimators = vote_est , voting = 'soft')
vote_soft_cv = model_selection.cross_validate(vote_soft, data1_x_bin, data[Target], cv  = skf,scoring='f1')
vote_soft.fit(data1_x_bin, data[Target])

#print("Soft Voting Training w/bin score mean: {:.2f}". format(vote_soft_cv['train_score'].mean()*100)) 
print("Soft Voting Test w/bin score mean: {:.2f}". format(vote_soft_cv['test_score'].mean()*100))
print("Soft Voting Test w/bin score 3*std: +/- {:.2f}". format(vote_soft_cv['test_score'].std()*100*3))
print('-'*10)z

## Stacking

#### ML Extend

In [0]:
from mlxtend.classifier import StackingClassifier
lgbm_cl = LGBMClassifier(random_state=seed)
rf_cl = RandomForestClassifier(10, random_state=seed)
gdb_cl = GradientBoostingClassifier(random_state=seed)
logreg = LogisticRegression()
sclf = StackingClassifier(classifiers=[lgbm_cl, rf_cl,gdb_cl],
                          meta_classifier=logreg)


scores = model_selection.cross_val_score(sclf, data1_x_bin, data[Target], 
                                              cv=3, scoring='f1')
print("Accuracy: %0.2f (+/- %0.2f)" 
      % (scores.mean(), scores.std()))

#### VecStack

In [0]:
#1st level model
X_train, X_test, y_train, y_test = train_test_split(data1_x_bin, data[Target], test_size=0.2)

models = [lgbm_cl,rf_cl,gdb_cl]
S_train, S_test = stacking(models, X_train, y_train, X_test, 
    regression = False, metric = metrics.f1_score, n_folds = 4 , 
    shuffle = True, random_state = 0, verbose = 2)



In [0]:
#2nd level model
# Initialize 2nd level model
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % metrics.f1_score(y_test, y_pred))

In [0]:
#New Experiment 

#Level 1
X_train, X_test, y_train, y_test = train_test_split(data1_x_bin, data[Target], test_size=0.2)

models = [
    ExtraTreesClassifier(random_state=0, n_jobs=-1, 
                         n_estimators=100, max_depth=3),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3)
]

S_train, S_test = stacking(models,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=metrics.f1_score,      # metric: callable
                           n_folds=4,                  # number of folds
                           stratified=True,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)      

In [0]:
#2nd level model
# Initialize 2nd level model
# model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
#                       n_estimators=100, max_depth=3)
    
model = LogisticRegression()
# model = KNeighborsClassifier(n_neighbors=5)
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % metrics.f1_score(y_test, y_pred))

# Automated Machine Learning (AutoML)

- Automated machine learning is the process of automating end-to-end the process of applying machine learning to real-world problems.

## TPot

In [0]:
#100 pipelines
#5 generations
# Create a tpot object with a few parameters
tpot = TPOTClassifier(generations=50,scoring ='f1', 
                    max_time_mins = 120, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 3)

# Convert to numpy arrays
training_features = np.array(X_train)


# Sklearn wants the labels as one-dimensional vectors
training_targets = np.array(y_train).reshape((-1,))

# Fit the tpot model on the training data
tpot.fit(training_features, training_targets)

In [0]:
# Show the final model
print(tpot.fitted_pipeline_)

In [0]:
# Evaluate the final model
print(tpot.score(X_test, y_test))

## MLBox

In [0]:
paths = ["train.csv",
         "test.csv"]
target_name = "Column_name_of_the_target_feaure"

In [0]:
rd = Reader(sep = ',')
df = rd.train_test_split(paths, target_name)

In [0]:
dft = Drift_thresholder()
df = dft.fit_transform(df)

In [0]:
opt = Optimiser(scoring = 'accuracy', n_folds = 3)
opt.evaluate(None, df)

In [0]:
space = {
    
        'est__strategy':{"search":"choice",
                                  "space":["LightGBM"]},    
        'est__n_estimators':{"search":"choice",
                                  "space":[150]},    
        'est__colsample_bytree':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__subsample':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__max_depth':{"search":"choice",
                                  "space":[5,6,7,8,9]},
        'est__learning_rate':{"search":"choice",
                                  "space":[0.07]} 
    
        }

best = opt.optimise(space, df,15)

In [0]:
prd = Predictor()
prd.fit_predict(best, df)

## H20

In [0]:
print(h2o.__version__)


h2o.init(max_mem_size='16G')

In [0]:
%%time
train = h2o.import_file("train.csv")
train = train[:250000,:]

In [0]:
x = train.columns[:-1]
y = 'Column_name_of_the_target_feaure'
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [0]:
aml = H2OAutoML(max_models=30, seed=45, max_runtime_secs=7200)
aml.train(x=x, y=y, training_frame=train)

In [0]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [0]:
# The leader model is stored here
aml.leader

In [0]:
h2o.remove(train)
test = h2o.import_file("test.csv")

In [0]:
h20_pred = aml.predict(test)

# Creating API

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data1_x_bin, data[Target], test_size=0.2)

lr = LogisticRegression()
lr.fit(X_train, y_train)

joblib.dump(lr, 'model.pkl')

In [0]:
# Loading Model 

lr = joblib.load('model.pkl')
print('Final prediction score: [%.8f]' % metrics.f1_score(y_test, lr.predict(X_test)))

In [0]:
data1_x_bin[:10].to_csv("testing_data.csv")

In [0]:
list(data1_x_bin[:10].columns)

In [0]:
str(lr.predict(data1_x_bin[:10]))

# Model Interpretation

In [0]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
dtree.predict(X_test)

## Eli5

In [0]:
perm = PermutationImportance(dtree , random_state=101).fit(X_test, y_test)      # Evaluate the permutation importance 
eli5.show_weights(perm, feature_names = X_test.columns.values)    

## Shap

In [0]:
row_to_show = 7                                                                # The row for which we want to check the SHAP explanations
data_to_predict = X_test.iloc[row_to_show]

data_to_preddict_array = data_to_predict.values.reshape(1,-1)

dtree_pred = dtree.predict_proba(data_to_preddict_array)  

dtree.predict(data_to_preddict_array)  

In [0]:
# Object that can calculate Shap values
explainer = shap.TreeExplainer(dtree)                                       # SHAP Tree Explainer

# Calculate the shap values
shap_values = explainer.shap_values(data_to_predict) 
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_to_predict)

## Lime

In [0]:
#Create an Explainer inhertance of the data
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=y_train.unique())

# Create a lambda function to use model to predict the data
predict_fn = lambda x: model.predict_proba(x).astype(float)

#using Explainer to Explain the predictions
exp = explainer.explain_instence(X_test.values[0], predict_fn, num_features=6)
exp.show_in_notebook(show_all=False)


## Yellowbrick

In [0]:
# HeatMap for Co-Relation

visualizer = Rank2D(algorithm="pearson", size=(1080, 720))
visualizer.fit_transform(X_train)
visualizer.poof()


# Evaluation Metrics
visualizer = ClassificationReport(model, size=(1080, 720))
visualizer.fit(X_train, y_train)
visualizer.score(X_train, y_train)
visualizer.poff()

# Using API in WebApp (Flask)

In [0]:
%%writefile server.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
from flask import Flask, request, jsonify
from sklearn.externals import joblib
import traceback
import pandas as pd
import numpy as np
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))


from flask import Flask, jsonify
app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
     json_ = request.get_json()
     lr = joblib.load("model.pkl")
     Columns = ['Title_1',
 'Sex_0',
 'Sex_1',
 'Pclass_3',
 'Pclass_1',
 'Title_2',
 'Title_3',
 'IsAlone_1',
 'Title_5',
 'FareBand_3']

    #  query_df = pd.DataFrame(json_)
     query_df = pd.read_csv("testing_data.csv",index_col=None)
     query_df = query_df[Columns]
    #  query = pd.get_dummies(query_df)
     print (query_df)
     prediction = lr.predict(query_df)
     return str(prediction)

if __name__ == '__main__':
    try:
        port = int(sys.argv[1]) # This is for a command-line input
    except:
        port = 12345 # If you don't provide any port the port will be set to 12345
    global lr
    lr = joblib.load("model.pkl") # Load "model.pkl"
    print ('Model loaded')
    # test_data = pd.read_csv("testing_data.csv")
    # return lr.predict(test_data)
    app.run(threaded=True,debug=True,port=8888)

In [0]:
import os
import subprocess
import sys
import time
import socket
# Start a subprocess that runs the Flask server
p = subprocess.Popen([sys.executable, "-m", "flask", "run"], env=dict(**os.environ, FLASK_APP="server.py"), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Start two subthreads that forward the output from the Flask server to the output of the Jupyter notebook
def forward(i, o):
    while p.poll() is None:
        l = i.readline().decode('utf-8')
        if l:
            o.write("[SERVER] " + l)

import threading
threading.Thread(target=forward, args=(p.stdout, sys.stdout)).start()
threading.Thread(target=forward, args=(p.stderr, sys.stderr)).start()
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))
# sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
# Let's give the server a bit of time to make sure it has started
time.sleep(2)

In [0]:
# Requesting to predict

import requests
# r = requests.get("http://172.28.0.2/")
r = requests.post(" http://127.0.0.1:5000/predict",data=data1_x_bin[:1].to_json())
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)

In [0]:
!ps -fA | grep python
!sudo kill 2230

#Happy Learning